# Griddap

Erddapy can access gridded datasets, using the server-side subsetting of griddap
or the OPeNDAP response, to download only the parts of a dataset that the user
requires.

In our example we will use a Region of Interest (ROI) to extract data within its
bounds. First we need to read the ROI with `geopandas`. Let's use the South
Atlantic Ocean basin from Natural Earth as our ROI.


In [ ]:
import geopandas
import pooch

url = "https://naturalearth.s3.amazonaws.com/4.1.1/50m_physical/ne_50m_geography_marine_polys.zip"
fname = pooch.retrieve(
    url,
    known_hash="db6f59e5a747c016451caec2450db6deea25d702dc2fb9c39384c1b909fb7f72",
)

oceans = geopandas.read_file(fname)

name = "South Atlantic Ocean"
SA = oceans.loc[oceans["name"] == name]

When accessing gridded datasets we need to define the `protocol="griddap"` in
our class instantiation.


In [ ]:
from erddapy import ERDDAP

e = ERDDAP(
    server="CSWC",  # CoastWatch West Coast Node
    protocol="griddap",  # Protocol for gridded datasets
)

e.dataset_id = "jplAvisoSshMon_LonPM180"  #  AVISO Model Output, obs4MIPs NASA-JPL, Global, 1 Degree

CAVEAT: Note that ERDDAP can serve gridded data with longitudes in the
0&ndash;360 format or -180&ndash;180. You must inspect the dataset and modify
your query accordingly.

Information on the griddap dataset is fetched with `griddap_initialize`. This
fills in the `variables` and `constraints` properties for that dataset.


In [ ]:
import json

e.griddap_initialize()

print(f"variables in this dataset:\n\n{e.variables}")
print(
    f"\nconstraints of this dataset:\n\n{json.dumps(e.constraints, indent=1)}"
)

The default behaviour is to use erddap standard subsetting: return all variables
at the most recent timestep and every point of the remaining dimensions.

This can result in large datasets, the values of the constraints can be changed,
and variables dropped before data set is downloaded. Here we will download only
one variable from that list.


In [ ]:
e.variables = [e.variables[0]]

print(f"Downloading {e.variables}.")

We will reduce the dataset a bit further by requesting only the data that is
inside the bounding box of the South Atlantic.


In [ ]:
def bounds2contraints(bounds):
    return {
        "longitude>=": bounds.minx.squeeze(),
        "longitude<=": bounds.maxx.squeeze(),
        "latitude>=": bounds.miny.squeeze(),
        "latitude<=": bounds.maxy.squeeze(),
    }


e.constraints.update(bounds2contraints(SA.bounds))
e.constraints

In [ ]:
print(f"\nconstraints for download:\n\n{json.dumps(e.constraints, indent=1)}")

Once the query is prepared we can download the data into an `xarray.Dataset`
object. (We can also download it in a `pandas.DataFrame` or `iris.Cube`
objects.)


In [ ]:
from urllib.parse import unquote_plus

url = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplAvisoSshMon_LonPM180.nc?sshag%5B(2010-12-16T12:00:00Z):1:(2010-12-16T12:00:00Z)%5D%5B(-60.53346241642455):1:(0.03286652261984102)%5D%5B(-69.09208207871731):1:(19.63485354989288)%5D,nObs%5B(2010-12-16T12:00:00Z):1:(2010-12-16T12:00:00Z)%5D%5B(-60.53346241642455):1:(0.03286652261984102)%5D%5B(-69.09208207871731):1:(19.63485354989288)%5D"

unquote_plus(url)

In [ ]:
%%time

ds = e.to_xarray()

Once downloaded, data can be quickly visualised with xarray's inbuilt plotting
functionality


In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ds["sshag"].plot(ax=ax)
ax.coastlines()

Note that we did not extract the exact ROI but instead we downloaded what is
inside its bounds. We can refine the data selection using region mask and
download strictly what is inside th ROI. The `regionmask` module can created
from the `geopandas` object.


In [ ]:
import regionmask


region = regionmask.from_geopandas(SA, name=name)
region.plot()

In [ ]:
mask = region.mask(
    ds,
    lon_name="longitude",
    lat_name="latitude",
    method="pygeos",
)


fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ds["sshag"].sel(time="2010").where(mask == region.numbers[0]).plot(ax=ax)
ax.coastlines()

The main difference is that now we did not load data from rivers plumes that are
not part of the ROI. (Check the Río de la Plata area in both plots.)


### Subset after the request with OPeNDAP

ERDDAP server-side subsetting can be avoided by specifying the OPeNDAP protocol.
This is a good choice if you intend to use a full dataset or subset it after the
request.

Note that most OPeNDAP clients will eagerly download only the coordinates,
making a post request subset almost as fast as serve-side subset.


In [ ]:
e = ERDDAP(
    server="CSWC",  # CoastWatch West Coast Node
    protocol="griddap",
    response="opendap",
)
e.dataset_id = "jplAquariusSSS3MonthV5"  # Aquarius Sea Surface Salinity, L3 SMI, Version 5, 1.0°, Global,

The data can be downloaded immediately, no need to run `griddap_initialize`


In [ ]:
%%time

ds = e.to_xarray()

Let's take a quick look at the data from 2015.


In [ ]:
projection = ccrs.PlateCarree()

fig, ax = plt.subplots(subplot_kw={"projection": projection})
sss = ds["sss"].sel(time="2015")
sss.plot(ax=ax)
ax.coastlines()

In [ ]:
mask = region.mask(
    ds,
    lon_name="longitude",
    lat_name="latitude",
    method="pygeos",
)


fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ds["sss"].sel(time="2015").where(mask == region.numbers[0]).plot(ax=ax)
ax.coastlines()

Thanks to `regionmask` we can go a few steps further with the ROI and compute
statistics. Let's calculate the mean salinity time-series for the South Atlantic
and the mean spatial salinity for the time coverage.


In [ ]:
sasal = ds.groupby(mask).mean()
sasal["sss"].plot(marker="o")

In [ ]:
sasal = ds.groupby(mask).mean(dim="time")
sasal["sss"].plot()